## Streaming Application

The streaming application generates a stream to accept continuos data records from producers. 


### Importing Libraries and Environment Variables

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.0 pyspark-shell'
import sys
import time
from datetime import datetime
import json
from pymongo import MongoClient
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from pprint import pprint
import Geohash

### Getting geohash value (Task C.2.2)

Method to generate Geo Hash value. Gets the latitute and longitude of the record and gets the geohash value for the same. 

Returns a dictionary with the geohash as the key and the record as the value.

In [2]:
def getGeohash(data):
    dataDict = json.loads(data[1])
    # Get latitude and longitude
    getLat = float(dataDict['latitude'])
    getLon = float(dataDict['longitude'])
    # Employ the geohash algorithm
    locHash = Geohash.encode(getLat,getLon,precision=5)
    # Generate a new dictionary which sends the geohash and record as key-value.
    newRDD = {}
    newRDD[locHash] = data[1]
    return newRDD   

### Aggregate Producer 2 and 3 (Task C.2.3)

Method aggregates the confidence and surface temperature values for records for prodcuer 2 and 3.

Returns a list of both records.

In [3]:
def joinDataFor23(rdd2, rdd3):
    # Extracts the two records
    rddItem1 = rdd2[0]
    rddItem2 = rdd3[0]
    # Calculate average confidence
    conf = (int(rddItem1['confidence']) + int(rddItem2['confidence']))/2
    rddItem1['confidence'] = conf
    rddItem2['confidence'] = conf
    # Calculate average surface temperature
    temp = (int(rddItem1['surface_temperature_celcius']) + int(rddItem2['surface_temperature_celcius']))/2
    rddItem1['surface_temperature_celcius'] = temp
    rddItem2['surface_temperature_celcius'] = temp
    return [rddItem1, rddItem2]

In [4]:
def combineAndStoreDB(iter):
    # Initializing empty lists for producer types.
    producer1List = []
    producer2List = []
    producer3List = []
    # Iterates over the groups
    for records in iter:
        # Iterates over the grouped records
        for prod in records[1]:
            # Converts the record to a json dictionary
            value_dict = json.loads(prod)
            # Fetches the producer value
            producer = value_dict['Sender_ID']
            # Allocates the records according to the producer type into the lists initialised earlier
            if producer == 'Producer01':
                # Formats the data according to model
                value_dict['Date'] = datetime.strptime(value_dict['Created_Time'][:10], '%Y-%m-%d')
                value_dict['Hotspot_ID'] = None
                value_dict['Station'] = None
                producer1List.append(value_dict)
            elif producer == 'Producer02':
                # Formats the data according to model
                value_dict['Date'] = datetime.strptime(value_dict['Created_Time'][:10], '%Y-%m-%d')
                value_dict['Datetime'] = value_dict.pop('Created_Time', None)
                producer2List.append(value_dict)
            else:
                # Formats the data according to model
                value_dict['Date'] = datetime.strptime(value_dict['Created_Time'][:10], '%Y-%m-%d')
                value_dict['Datetime'] = value_dict.pop('Created_Time', None)
                producer3List.append(value_dict)
    
    # Gets lengths of the lists for comparison
    length1 = len(producer1List)
    length2 = len(producer2List)
    length3 = len(producer3List)
    # Initialises final lists for collections
    hotspot_data = []
    climate_data = []
    # If group contains records for all producers
    if length1 > 0 and length2 > 0 and length3 > 0:
        # Aggregates the data for Producer 2 and 3
        hotspot_data = joinDataFor23(producer2List, producer3List)
        climate_data = producer1List
    # If group contains only Producers 1 and 2
    elif length1 > 0 and length2 > 0:
        hotspot_data = producer2List
        climate_data = producer1List
    # If group contains only Producers 1 and 3
    elif length1 > 0 and length3 > 0:
        hotspot_data = producer3List
        climate_data = producer1List
    # If group contains only Producers 1 (Task C.2.4)
    elif length1 > 0:
        climate_data = producer1List
    
    # Sends collection lists for insertion into DB.
    sendDataToDB(climate_data, hotspot_data)
    return (climate_data, hotspot_data)    

### Insertion into DB

Inserts the data into their respective collections in the DB.

The hotspot(fires) data is inserted earlier as a list of ID's is required to link fires and climate data. Upon successful insertion of fire data, the obtained ID's are then added to the climate data as per the referential model designed.

In [5]:
def sendDataToDB(climate_data, hotspot_data):
    # Connects to the MongoClient
    client = MongoClient()
    # Connects to the db and collections
    db = client.fit5148_db
    climateCol = db.climate_historic
    hotspotCol = db.hotspot_historic
    try:
        # Checks if there is hotspot(fire) data. If there is then inserts into collection.
        if len(hotspot_data) > 0:
            # Inserting hotspot data
            hotspot_id = hotspotCol.insert(hotspot_data)
            # For each climate data entry enters the corresponding hotspot data
            for climate in climate_data:
                climate['Hotspot_ID'] = hotspot_id
        # Checks if there is climate data. If there is then inserts into collection.
        if len(climate_data) > 0:
            # Inserting climate data
            climateCol.insert(climate_data)
    except Exception as ex:
        print("Exception Occured. Message: {0}".format(str(ex)))
    client.close()

In [6]:
# #Drop Dbs if needed

# client = MongoClient()
#     # Connects to the db and collections
# db = client.fit5148_db
# db.climate_historic.drop()
# db.hotspot_historic.drop()

### Streaming (Task C.2.1)

Makes a local streaming context with two execution threads and a batch interval of 10 seconds. Streaming recieves data from all three producers in a single kafka stream. The stream is then mapped to generate a geohash value for each RDD element. The resulting transformed RDD is then split into key-value pairs where key is the geohash key and the value is the respective record. The transformed RDD is then grouped by the key and then each element is sent for further procfessing for DB insertion.

In [7]:
# Batch interval
n_secs = 10

# Spark configuration with two execution threads
conf = SparkConf().setAppName("KafkaStreamProcessor").setMaster("local[2]")
# Creating or getting the spark context
sc = SparkContext.getOrCreate()
if sc is None:
    sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
# Getting the Streaming context
ssc = StreamingContext(sc, n_secs)

# Kafka stream for getting the records from Producer 1,2 and 3
kafkaStream = KafkaUtils.createDirectStream(ssc, ["Producer"], {
                        'bootstrap.servers':'127.0.0.1:9092', 
                        'group.id':'lab27_group1', 
                        'fetch.message.max.bytes':'15728640',
                        'auto.offset.reset':'largest'})

# Geo hashing all RDD's in the stream
newRDDByGeoHash1 = kafkaStream.map(lambda x : getGeohash(x))

newRDDByGeoHash1.pprint()

# Makes key-value pairs for geo location as key and corresponding records as values
rdd1 = newRDDByGeoHash1.map(lambda rdd: (list(rdd.keys())[0], next(iter(rdd.values()))))

# Groups the transformed RDD by key (geolocation)
rddGroups = rdd1.groupByKey()

# Sends the geolocated groups for processing and insertion in DB
finalrdd = rddGroups.foreachRDD(lambda rdd: rdd.foreachPartition(combineAndStoreDB))

# Starts the stream
ssc.start()
# Run stream for 100 minutes just in case no detection of producer
time.sleep(6000) 
# Stops the stream
ssc.stop(stopSparkContext=True,stopGraceFully=True)

-------------------------------------------
Time: 2019-05-24 21:59:10
-------------------------------------------

-------------------------------------------
Time: 2019-05-24 21:59:20
-------------------------------------------
{'r1tys': '{"longitude": "143.267", "latitude": "-35.4033", "surface_temperature_celcius": "67", "confidence": "90", "Created_Time": "2019-05-24 21:59:10", "Sender_ID": "Producer03"}'}
{'r1kt1': '{"windspeed_knots": "8", "max_wind_speed": "16.9", "Sender_ID": "Producer01", "precipitation": "0.00I", "Created_Time": "2019-05-24 21:59:11", "longitude": "141.373", "latitude": "-37.062", "relative_humidity": "46.8", "air_temperature_celcius": "22"}'}
{'r1mx0': '{"windspeed_knots": "12.9", "max_wind_speed": "19", "Sender_ID": "Producer01", "precipitation": "0.02G", "Created_Time": "2019-05-24 21:59:16", "longitude": "142.7354", "latitude": "-36.7084", "relative_humidity": "44.1", "air_temperature_celcius": "13"}'}

-------------------------------------------
Time: 20

-------------------------------------------
Time: 2019-05-24 22:01:00
-------------------------------------------
{'r1w4w': '{"windspeed_knots": "4.2", "max_wind_speed": "7", "Sender_ID": "Producer01", "precipitation": "0.01G", "Created_Time": "2019-05-24 22:00:51", "longitude": "143.74", "latitude": "-36.098", "relative_humidity": "59.3", "air_temperature_celcius": "17"}'}
{'r1s76': '{"windspeed_knots": "4.3", "max_wind_speed": "11.1", "Sender_ID": "Producer01", "precipitation": "0.00A", "Created_Time": "2019-05-24 22:00:56", "longitude": "141.089", "latitude": "-35.961", "relative_humidity": "34.9", "air_temperature_celcius": "5"}'}
{'r365m': '{"longitude": "149.311", "latitude": "-37.389", "surface_temperature_celcius": "50", "confidence": "73", "Created_Time": "2019-05-24 22:00:59", "Sender_ID": "Producer03"}'}

-------------------------------------------
Time: 2019-05-24 22:01:10
-------------------------------------------
{'r1km4': '{"windspeed_knots": "6.9", "max_wind_speed": "9

-------------------------------------------
Time: 2019-05-24 22:02:50
-------------------------------------------
{'r1mfy': '{"windspeed_knots": "9.4", "max_wind_speed": "25.1", "Sender_ID": "Producer01", "precipitation": "0.16G", "Created_Time": "2019-05-24 22:02:42", "longitude": "143.351", "latitude": "-37.467", "relative_humidity": "53.4", "air_temperature_celcius": "17"}'}
{'r1m2s': '{"longitude": "142.5842", "latitude": "-37.8704", "surface_temperature_celcius": "48", "confidence": "74", "Created_Time": "2019-05-24 22:02:42", "Sender_ID": "Producer03"}'}
{'r3372': '{"windspeed_knots": "5.5", "max_wind_speed": "8", "Sender_ID": "Producer01", "precipitation": "0.00G", "Created_Time": "2019-05-24 22:02:47", "longitude": "148.042", "latitude": "-37.376", "relative_humidity": "43.2", "air_temperature_celcius": "11"}'}
{'r1tys': '{"confidence": "90", "Sender_ID": "Producer02", "latitude": "-35.4033", "longitude": "143.267", "Created_Time": "2019-05-24 22:02:47", "surface_temperature_ce

-------------------------------------------
Time: 2019-05-24 22:04:30
-------------------------------------------
{'r1s3p': '{"windspeed_knots": "9.5", "max_wind_speed": "13", "Sender_ID": "Producer01", "precipitation": "0.08G", "Created_Time": "2019-05-24 22:04:22", "longitude": "141.3146", "latitude": "-36.3823", "relative_humidity": "43.7", "air_temperature_celcius": "10"}'}
{'r1w1h': '{"windspeed_knots": "4.9", "max_wind_speed": "13", "Sender_ID": "Producer01", "precipitation": "0.00I", "Created_Time": "2019-05-24 22:04:27", "longitude": "143.631", "latitude": "-36.346", "relative_humidity": "47.2", "air_temperature_celcius": "17"}'}
{'r1mbn': '{"confidence": "68", "Sender_ID": "Producer02", "latitude": "-37.9309", "longitude": "143.381", "Created_Time": "2019-05-24 22:04:29", "surface_temperature_celcius": "44"}'}

-------------------------------------------
Time: 2019-05-24 22:04:40
-------------------------------------------
{'r1mrv': '{"windspeed_knots": "5.5", "max_wind_speed"

-------------------------------------------
Time: 2019-05-24 22:06:10
-------------------------------------------
{'r336f': '{"windspeed_knots": "11.2", "max_wind_speed": "16.9", "Sender_ID": "Producer01", "precipitation": "0.12G", "Created_Time": "2019-05-24 22:06:02", "longitude": "148.117", "latitude": "-37.478", "relative_humidity": "43.9", "air_temperature_celcius": "11"}'}
{'r1x4p': '{"longitude": "145.1836", "latitude": "-36.1841", "surface_temperature_celcius": "52", "confidence": "79", "Created_Time": "2019-05-24 22:06:03", "Sender_ID": "Producer03"}'}
{'r1mg8': '{"windspeed_knots": "7.8", "max_wind_speed": "15.9", "Sender_ID": "Producer01", "precipitation": "0.00I", "Created_Time": "2019-05-24 22:06:07", "longitude": "143.122", "latitude": "-37.331", "relative_humidity": "44.2", "air_temperature_celcius": "19"}'}

-------------------------------------------
Time: 2019-05-24 22:06:20
-------------------------------------------
{'r1t99': '{"windspeed_knots": "5.7", "max_wind_sp

...

-------------------------------------------
Time: 2019-05-24 22:42:10
-------------------------------------------
{'r1w1n': '{"windspeed_knots": "8.5", "max_wind_speed": "12", "Sender_ID": "Producer01", "precipitation": "0.08G", "Created_Time": "2019-05-24 22:42:00", "longitude": "143.7132", "latitude": "-36.369", "relative_humidity": "50.1", "air_temperature_celcius": "13"}'}
{'r3373': '{"windspeed_knots": "6.5", "max_wind_speed": "12", "Sender_ID": "Producer01", "precipitation": "0.02G", "Created_Time": "2019-05-24 22:42:05", "longitude": "148.052", "latitude": "-37.365", "relative_humidity": "43.8", "air_temperature_celcius": "11"}'}
{'r30pw': '{"longitude": "146.5413", "latitude": "-38.0148", "surface_temperature_celcius": "71", "confidence": "87", "Created_Time": "2019-05-24 22:42:06", "Sender_ID": "Producer03"}'}
{'r1w4z': '{"confidence": "69", "Sender_ID": "Producer02", "latitude": "-36.059", "longitude": "143.7718", "Created_Time": "2019-05-24 22:42:07", "surface_temperature_ce

-------------------------------------------
Time: 2019-05-24 22:43:50
-------------------------------------------
{'r1x62': '{"windspeed_knots": "5.5", "max_wind_speed": "11.1", "Sender_ID": "Producer01", "precipitation": "0.00G", "Created_Time": "2019-05-24 22:43:40", "longitude": "145.2196", "latitude": "-36.1346", "relative_humidity": "37.2", "air_temperature_celcius": "8"}'}
{'r1xct': '{"windspeed_knots": "5.8", "max_wind_speed": "12", "Sender_ID": "Producer01", "precipitation": "0.00I", "Created_Time": "2019-05-24 22:43:45", "longitude": "146.148", "latitude": "-36.293", "relative_humidity": "50.8", "air_temperature_celcius": "16"}'}

-------------------------------------------
Time: 2019-05-24 22:44:00
-------------------------------------------
{'r1t2u': '{"windspeed_knots": "10.7", "max_wind_speed": "22.9", "Sender_ID": "Producer01", "precipitation": "0.67G", "Created_Time": "2019-05-24 22:43:50", "longitude": "142.5598", "latitude": "-36.4025", "relative_humidity": "57.4", "ai

-------------------------------------------
Time: 2019-05-24 22:45:30
-------------------------------------------
{'r361u': '{"windspeed_knots": "8.7", "max_wind_speed": "15", "Sender_ID": "Producer01", "precipitation": "0.00I", "Created_Time": "2019-05-24 22:45:20", "longitude": "149.261", "latitude": "-37.627", "relative_humidity": "53.7", "air_temperature_celcius": "26"}'}
{'r364j': '{"windspeed_knots": "10", "max_wind_speed": "19", "Sender_ID": "Producer01", "precipitation": "0.00I", "Created_Time": "2019-05-24 22:45:25", "longitude": "149.326", "latitude": "-37.605", "relative_humidity": "49.6", "air_temperature_celcius": "23"}'}
{'r1q0h': '{"longitude": "143.6199", "latitude": "-37.9422", "surface_temperature_celcius": "39", "confidence": "53", "Created_Time": "2019-05-24 22:45:26", "Sender_ID": "Producer03"}'}

-------------------------------------------
Time: 2019-05-24 22:45:40
-------------------------------------------
{'r1mdx': '{"windspeed_knots": "14.4", "max_wind_speed":

KeyboardInterrupt: 

-------------------------------------------
Time: 2019-05-24 22:46:20
-------------------------------------------
{'r1w1n': '{"windspeed_knots": "16.1", "max_wind_speed": "27", "Sender_ID": "Producer01", "precipitation": "0.24G", "Created_Time": "2019-05-24 22:46:10", "longitude": "143.7313", "latitude": "-36.3782", "relative_humidity": "44.9", "air_temperature_celcius": "17"}'}
{'r364j': '{"windspeed_knots": "9", "max_wind_speed": "15.9", "Sender_ID": "Producer01", "precipitation": "0.00G", "Created_Time": "2019-05-24 22:46:15", "longitude": "149.295", "latitude": "-37.602", "relative_humidity": "50.8", "air_temperature_celcius": "17"}'}

-------------------------------------------
Time: 2019-05-24 22:46:30
-------------------------------------------
{'r364j': '{"windspeed_knots": "10.9", "max_wind_speed": "19", "Sender_ID": "Producer01", "precipitation": "0.00I", "Created_Time": "2019-05-24 22:46:20", "longitude": "149.297", "latitude": "-37.6", "relative_humidity": "53.4", "air_temp